## Set-up

In [ ]:
# Environment setup
from google.colab import drive
import pandas as pd
import numpy as np
!pip3 install num2words
from num2words import num2words
import re
import os
import time
!pip3 install contractions
import contractions
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=9ac77a5f52860d3d148781f338e3224e224d33f230c76a2c7e601baad4e7f9ed
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 26.2 MB/s eta 0:00:00


In [ ]:
# Mount GoogleDrive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load data

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/SciFi/internet_archive_scifi_v3.txt','r', encoding="utf-8") as f:
  scifi = f.read()

num_chars = len(scifi)
print("Number of characters in the document:", num_chars)

Number of characters in the document: 149326361


In [ ]:
# The unique characters in the file
vocab = sorted(set(scifi))
print(f'{len(vocab)} unique characters')

75 unique characters


In [ ]:
# Extract the first 5500 characters of the row
first_5500_chars = scifi[:5500]
print(first_5500_chars) # Remove the first 5200 as copyright materical and not scifi stories.

MARCH # All Stories New and Complete Publisher Editor IF is published bi-monthly by Quinn Publishing Company, Inc., Kingston, New York. Volume #, No. #. Copyright # by Quinn Publishing Company, Inc. Application for Entry' as Second Class matter at Post Office, Buffalo, New York, pending. Subscription # for # issues in U.S. and Possessions: Canada # for # issues; elsewhere #. Aiiow four weeks for change of address. All stories appearing in this magazine are fiction. Any similarity to actual persons is coincidental. #c a fcopy. Printed ia U.S. A. A chat with the editor  i #  science fiction magazine called IF. The title was selected after much thought because of its brevity and on the theory it is indicative of the field and will be easy to remember. The tentative title that just morning and couldn't remember it until we'd had a cup of coffee, it was summarily discarded. A great deal of thought and effort lias gone into the formation of this magazine. We have had the aid of several very 

In [ ]:
# Find how many times 'pwf' appear
print(scifi.find('pwf')) # Remove

5196


In [ ]:
# Extract the last 2000 characters
last_2000_chars = scifi[-2000:]
print(last_2000_chars) # Remove the last 1944 as not scifi stories.

tury undersea mission from which no human has ever returned. Now, # feet down, you learn that an (Publ. ed. #) Ed. by T. E. Dikty # top-notch short stories and novelettes by such well known writers as Robert Bloch. Frank Robinson, Mark Clifton, many others. (Publ. ed. #) Just Mail Coupon In.ll.'i.l ' mi coupon your choice of waul in inhlit imi to "Satellite!" One bool, will lie considered your first i led ion for which you'll be billed only I plus a few cents jnon I I I you will be offered the cream of I lie new # to # science lid ion books -- for only I each. You take only those books you rcnlly want -- as few as four a year llut this offer may be with. I I a w I I at any time. So mail Del OX-#, Garden City, N. Y. Dept. GX-#, Garden City, N. Y. Enroll me as a member, and rush me my full-length handcated below. Bill me only I (plus few cents shipping charges). Every month send me the Club's free bulletin, "Things to Come," so that I may decide whether or not I wish to receive the coini

In [ ]:
# Define function to calculate average sentence length
def average_sentence_len(text):
  sentences = text.split(".")
  words = text.split(" ")
  if (sentences[len(sentences)-1]==""):
    average_sentence_length= len(words) / len(sentences)-1
  else:
    average_sentence_length = len(words) / len(sentences)

  return average_sentence_length

# Define function to calculate maximum sentence length
def max_sentence_len(text):
  sentences = text.split(".")
  max_len = max([len(sentence.split()) for sentence in sentences])
  return max_len

In [ ]:
# Get max and average sentence length
average_sentence_length = average_sentence_len(scifi)
max_sentence_length = max_sentence_len(scifi)
print(f"Average sentence length: {average_sentence_length}")
print(f"Maximum sentence length: {max_sentence_length}")

Average sentence length: 12.553956848085688
Maximum sentence length: 326


In [ ]:
# Count how many words are in the text
word_count = len(scifi.split(" "))
print("The number of words in the sample text is:", word_count)

The number of words in the sample text is: 26655101


## Preprocessing

In [ ]:
# Define a function to convert numbers to text
def convert_numbers_to_text(text):
    numbers = re.findall(r'\d+', text)
    for number in numbers:
        text = text.replace(number, num2words(int(number)))
    return text

In [ ]:
# Define function to clean and preprocess the text
def preprocess(text):
  text = text.replace(text[:5200], '') # Delete start of dataset
  text = text.replace(text[-1994:], '') # Delete end of dataset
  text = text.replace('pwf', '')
  text = convert_numbers_to_text(text)
  text = text.replace('&', 'and')
  text = text.replace('%', ' percent')
  text = contractions.fix(text)
  # A list of special charaters to be removed
  special_characters = ['@', '#', '$', '£', '*', '-', '+', '=', '~', '<', '>']
  for i in special_characters:
    if i != ',' or '.':
      text = text.replace(i, "")
  return text

In [ ]:
# Preprocess the original text
clean_scifi = preprocess(scifi)

In [ ]:
# Save the preprocessed text to a file
with open('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/SciFi/clean_scifi.txt', 'w', encoding='utf-8') as f:
  f.write(clean_scifi)

In [ ]:
# Load preprocessed data
with open('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/SciFi/clean_scifi.txt','r', encoding="utf-8") as f:
  clean_scifi = f.read()

In [ ]:
# Extract the first 300 characters to check it's correct
first_300_chars = clean_scifi[:300]
print(first_300_chars)

 Police grilled him mercilessly  while eyes from a hundred worlds looked on  It was a lovetriangle murder that made today's headlines but the answer lay hundreds of thousands of light years away! one of the basement rooms. He moved slowly and with a kind of painful dignity, as a man moves on his way


In [ ]:
# Get max and average sentence length
average_sentence_length = average_sentence_len(clean_scifi)
max_sentence_length = max_sentence_len(clean_scifi)
print(f"Average sentence length: {average_sentence_length}")
print(f"Maximum sentence length: {max_sentence_length}")

Average sentence length: 12.755057726536492
Maximum sentence length: 321


In [ ]:
# The unique characters in the file
clean_vocab = sorted(set(clean_scifi))
print(f'{len(clean_vocab)} unique characters')

63 unique characters


In [ ]:
# Count how many words are in the text
clean_word_count = len(clean_scifi.split(" "))
print("The number of words in the text is:", clean_word_count)

The number of words in the text is: 27073809


In [ ]:
# Count unique words

def countUniqueWords(preprocessed_text):
    words_in_file = preprocessed_text.split(" ")
    count_map = {}
    for i in words_in_file:
        if i in count_map:
            count_map[i] += 1
        else:
            count_map[i] = 1
    count = 0
    for i in count_map:
        if count_map[i] == 1:
            count += 1
    return count

# Assume that clean_scifi contains the preprocessed text
unique_words_count = countUniqueWords(clean_scifi)
print('Number of unique words in the file are:', unique_words_count)

Number of unique words in the file are: 385901


In [ ]:
import pickle
!pip install pyyaml h5py
import h5py

def save_tokenizer(tokenizer, filename):
    with h5py.File(filename, 'w') as f:
        for key, value in tokenizer.word_index.items():
            f[key] = np.array([value])

def load_tokenizer(filename):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000)
    with h5py.File(filename, 'r') as f:
        tokenizer.word_index = {key: int(value[()]) for key, value in f.items()}
    return tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
type(clean_scifi)

str

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Instantiate the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_scifi.split(' ')[:20000])

#save_tokenizer(tokenizer, '/content/gdrive/MyDrive/Colab Notebooks/CSC8637/checkpoints/tokenizer.h5')

In [ ]:
# Load tokenizer
#tokenizer = load_tokenizer('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/checkpoints/tokenizer.h5')


In [ ]:
# Set the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

3528


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a function to convert the text into sequences and pad them
def tokenize_and_pad(text):
    # Convert the text into sequences
    sequences = tokenizer.texts_to_sequences(text)
    # Pad the sequences
    padded_sequences = pad_sequences(sequences, maxlen=21, padding='pre')
    return padded_sequences

# def generate_batches(text, token_batch_size):
#   seq, labels = [], []
#   for i in range(0, len(text), token_batch_size):
#     for j in range(i, i+token_batch_size):
#       chunk = text[i:j+2]
#       sequences = tokenize_and_pad(chunk)
#       #yield sequences[:,:-1], keras.utils.to_categorical(sequences[:,-1], num_classes = vocab_size)
#       seq.append([sequences[:-1]])
#       labels.append(keras.utils.to_categorical(sequences[-1], num_classes = vocab_size))
#   return seq, labels

def generate_batches(text, token_batch_size):
  sequences = np.array(tokenize_and_pad(text))
  seq, labels = sequences[:,:-1], sequences[:,-1]
  labels = keras.utils.to_categorical(labels, num_classes=vocab_size)
  return seq, labels

In [ ]:
token_batch_size = 20

# Tokenize and pad the train, validation, and test text
generated_text = generate_batches(clean_scifi[:8000].split(' '), token_batch_size)

sequences, labels = generated_text

## Modelling

In [ ]:
# Build model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=20))
model.add(LSTM(640, return_sequences=True))
model.add(Dense(1280, activation="relu"))
model.add(Flatten())
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
# Summarize
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 10)            35280     
                                                                 
 lstm (LSTM)                 (None, 20, 640)           1666560   
                                                                 
 dense (Dense)               (None, 20, 1280)          820480    
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense_1 (Dense)             (None, 3528)              90320328  
                                                                 
Total params: 92,842,648
Trainable params: 92,842,648
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile and fit model
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), run_eagerly=True)

model.fit(sequences, labels, epochs=20, batch_size=64, steps_per_epoch=100)


Epoch 1/20


100/100 [==============================] - 15s 78ms/step - loss: 6.1416
Epoch 2/20
100/100 [==============================] - 6s 58ms/step - loss: 5.7524
Epoch 3/20
100/100 [==============================] - 5s 53ms/step - loss: 5.7298
Epoch 4/20
100/100 [==============================] - 6s 61ms/step - loss: 5.7196
Epoch 5/20
 80/100 [=======================>......] - ETA: 1s - loss: 5.7202

100/100 [==============================] - 4s 43ms/step - loss: 5.7202


In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format
model.save('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/lstm.h5')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Generate Text

In [ ]:
# Load the model
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/lstm.h5')

In [ ]:
# Define a function to generate text

def generate_text(model, tokenizer, seed_sequence, num_words):
    """
    Generate text using the specified model, tokenizer, seed sequence, and number of words to generate.

    Args:
    - model: The trained Keras model to use for text generation.
    - tokenizer: The Keras Tokenizer used to convert text to sequences.
    - seed_sequence: The seed sequence to use for text generation, as a string.
    - num_words: The number of words to generate after the seed sequence.

    Returns:
    - The generated text as a string.
    """
    # Convert the seed sequence to a sequence of integer tokens
    #seed_tokens = tokenizer.texts_to_sequences([seed_sequence])[0]

    # Generate num_words additional tokens using the model
    probs_arr = []
    for i in range(num_words):
        seed_tokens = tokenizer.texts_to_sequences([seed_sequence])
        padded_seed = pad_sequences(seed_tokens, maxlen=20, padding='pre')

        # Predict the next token(s) given the current sequence
        predicted_probs = model.predict(padded_seed)

        output_word = ""
        for word,index in tokenizer.word_index.items():
        #print(f'word:{word}, index:{index}')
          if index == np.argmax(predicted_probs, axis=-1):
            output_word = word
            break
        seed_sequence += " "+output_word

    return seed_sequence


In [ ]:
#Example prompts:
#My only thought was to hide
#She shook her head emphatically
#handle it wisely
#At first, of course, you will rebel.
#I was smart enough to fool all the so-called brains of the Solar System


# Generate text using the model and tokenizer
seed_sequence = "I was smart enough to fool all the so-called brains of the Solar System."
num_words = 20
generated_text = generate_text(model, tokenizer, seed_sequence=seed_sequence, num_words=num_words)

# Print the generated text
print(generated_text)

1/1 [==============================] - 0s 41ms/step
I was smart enough to fool all the so-called brains of the Solar System. the the the the the the the the the the the the the the the the the the the the
